In [172]:
import os
import re
import random
import glob
import importlib
import sys
from bs4 import BeautifulSoup
import urllib
import urllib.request 
from urllib.request import urlopen
from urllib.request import Request
#from urllib2 import Request


import requests as req
from collections import defaultdict
import time

import pandas as pd
import numpy as np

import base64
import io

# visualization imports
from IPython.display import display
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline
sns.set() 

import ipywidgets as widgets
from ipywidgets import interact, interact_manual

from pdfminer.pdfparser import PDFParser
from pdfminer.pdfdocument import PDFDocument
from pdfminer.pdfpage import PDFPage
from pdfminer.pdfpage import PDFTextExtractionNotAllowed
from pdfminer.pdfinterp import PDFResourceManager
from pdfminer.pdfinterp import PDFPageInterpreter
from pdfminer.layout import LAParams 
from pdfminer.converter import TextConverter


from sklearn import metrics
from sklearn.decomposition import PCA, LatentDirichletAllocation
from sklearn.preprocessing import scale, StandardScaler
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer,TfidfTransformer


import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 
from nltk.tokenize import sent_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.probability import ConditionalFreqDist 
from nltk.probability import FreqDist 
nltk.download('stopwords')
nltk.download('punkt')
set(stopwords.words('spanish'))
nltk.download('wordnet')



[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/aideenf/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /Users/aideenf/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/aideenf/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [173]:
%%javascript
//To remove scroll from any output area and automatically extend the jupyter cell
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

# Import the helper class and methods

In [174]:
#We are using helper methods from this file 
%run ./AIVM_helper_classes.ipynb
#from this class
helper =  AIVM_Generic_helper()

Initialised AIVM helper class


set up global variables

In [175]:
LOCAL_PDF_DOCS_DIR =  './Data/Iterative-models-building/Gathered_data/Conventions/pdf_docs/'
if not os.path.exists(LOCAL_PDF_DOCS_DIR):
    os.makedirs(LOCAL_PDF_DOCS_DIR)

#Classify and audit path goes to the location where the classifier reads the audit files
CLAS_AUDIT_PATH = 'cp_wssc/Data/Iterative-models-building/Gathered_data/Conventions/'

#train path can point straight to the location where we push the already audited files
TRAIN_PATH = 'cp_wssc/Data/Iterative-models-building/Classification results/Conventions/Audited/'

#Github information for reading files directly from github
GIT_OWNER = 'aideenf'
GIT_REPO = 'AIVC'    
NUM_KEYWORDS_PER_CONV = 30

# read pdf files to process directly from github.
to obtain the id for the folder, navigate the tree using
https://api.github.com/repos/{owner}/{repo}/git/trees/master
e.g https://api.github.com/repos/aideenf/AIVC/git/trees/master
once navigated each directory will be of format
https://api.github.com/repos/aideenf/AIVC/git/trees/{dir_sha}
eg. https://api.github.com/repos/aideenf/AIVC/git/trees/392e7c49c4fc5825a54ab8c369997cfcd99b5fd0

In [176]:
gathered_pdf_files_list = []
gathered_pdf_files_dict = {}
pdf_list = []

dir_path = 'cp_wssc/Data/Iterative-models-building/Gathered_data/Conventions/pdf_docs'
gathered_pdf_files_list  = helper.list_files_from_github_dir(GIT_OWNER, GIT_REPO, dir_path)   

In [177]:
gathered_pdf_files_list =  helper.remove_excluded_files_except(gathered_pdf_files_list, ".pdf")

#Read pdf  file from github.
for f in gathered_pdf_files_list:
    urlBase  ='https://github.com/aideenf/AIVC/raw/master/cp_wssc/Data/Iterative-models-building/Gathered_data/Conventions/pdf_docs/'
    gathered_pdf_files_dict[f] = urlBase + f


# Create a widget to allow for the review and selection of the pdf to scan

In [178]:
my_style = {'description_width': 'initial'}
my_layout = {'width': '500px'}
my_layout_short = {'width': '300px'}
convention_drop_down = widgets.Dropdown(
    options=['unknown','civic', 'domestic', 'green', 'industrial', 'inspired', 'market', 'project', 'renown'],
    value='unknown',
    description='Convention:',
    disabled=False,
    style = my_style,
    layout = my_layout
)

pdf_drop_down = widgets.Dropdown(
    options =  gathered_pdf_files_dict,
    description='File to parse:',
     #value=None,
    disabled=False,
    style = my_style,
    layout = my_layout
)

git_user = widgets.Text(
            description='GIT username',
            disabled=False,
            layout=my_layout_short,
            tooltip='enter the new value name',
            style = my_style
        )


git_pswd = widgets.Password(
            description='GIT password',
            disabled=False,
            layout=my_layout_short,
            tooltip='enter the new value name',
            style = my_style
        )


def output_function(convention_drop_down_value, pdf_drop_down_value, git_user_value, git_pswd_value ):
    print (convention_drop_down_value)
    print (pdf_drop_down_value)
    print (git_user_value)
    

output = widgets.interactive_output(output_function, {
            'convention_drop_down_value': convention_drop_down,
            'pdf_drop_down_value': pdf_drop_down,
            'git_user_value': git_user,
            'git_pswd_value': git_pswd
            })

display (convention_drop_down, pdf_drop_down, git_user, git_pswd, output)

Dropdown(description='Convention:', layout=Layout(width='500px'), options=('unknown', 'civic', 'domestic', 'gr…

Dropdown(description='File to parse:', layout=Layout(width='500px'), options={'AIFORSUSTAINABLEINTELIGENTBUILD…

Text(value='', description='GIT username', layout=Layout(width='300px'), style=DescriptionStyle(description_wi…

Password(description='GIT password', layout=Layout(width='300px'), style=DescriptionStyle(description_width='i…

Output()

# Read the pdf

In [179]:
url = pdf_drop_down.value
print (url)
#response = urllib.request.urlopen(Request(url)).read()
response = urllib.request.urlopen(url).read()
print (type (response))
memoryFile = io.BytesIO(response)
parser = PDFParser(memoryFile)
document = PDFDocument(parser)

# Try to parse the document
if not document.is_extractable:
    raise PDFTextExtractionNotAllowed

# Create a PDF resource manager object 
# that stores shared resources.
rsrcmgr = PDFResourceManager()

# Create a buffer for the parsed text
retstr = io.StringIO()

# Spacing parameters for parsing
laparams = LAParams()
codec = 'utf-8'
 
# Create a PDF device object
device = TextConverter(rsrcmgr, retstr, 
                        codec = codec, 
                        laparams = laparams)
# Create a PDF interpreter object
interpreter = PDFPageInterpreter(rsrcmgr, device)

# Process each page contained in the document.
for page in PDFPage.create_pages(document):
    interpreter.process_page(page)

print(type(document))
#tokenised_into_sentences = sent_tokenize(document)
#print(tokenised_into_sentences)
         
records = []

myStr = ""       
lines = retstr.getvalue().splitlines()
for line in lines:
    records.append(line)
    myStr = myStr + line

    


https://github.com/aideenf/AIVC/raw/master/cp_wssc/Data/Iterative-models-building/Gathered_data/Conventions/pdf_docs/Harnessing_Artificial_Intelligence_for_the_Earth_report_2018.pdf
<class 'bytes'>
<class 'pdfminer.pdfdocument.PDFDocument'>


In [180]:
texts = []
data_provenance = []
convention = []
confidence_value = []
confidence_level = []
old = []
new = []

#tokenised_into_sentences = sent_tokenize(myStr)
tokenised_into_sentences = helper.pre_process(myStr)
print ("number of sentences from pdf: ", len(tokenised_into_sentences))


for s in tokenised_into_sentences:
    if len(s) > 25 and len(s) < 500 and not "....." in s and not "download here citation" in s:
       
        s = helper.pre_process_sentence(s)
        print("******NEXT*******")
        print(s)
        texts.append(str(s))
        data_provenance.append ("pdf")
        convention.append (convention_drop_down.value)
        confidence_value.append (0)
        confidence_level.append ("not-from-model")
        old.append (1)
        new.append (1)
                
print ("Final number of sentences after processing: ", len(texts))

number of sentences from pdf:  359
******NEXT*******
fourth industrial revolution for the earth seriesharnessing artificial intelligence for the earthin collaboration with pwc and stanford woods institute for the environmentjanuary harnessing artificial intelligence for the earth is published by the world economic forum system initiative on shaping the future of environment and natural resource security in partnership with pwc and the stanford woods institute for the environment.
******NEXT*******
it was made possible with funding from the mava foundation.about the fourth industrial revolution for the earth series the fourth industrial revolution for the earth is a publication series highlighting opportunities to solve the world s most pressing environmental challenges by harnessing technological innovations supported by new and effective approaches to governance financing and multistakeholder collaboration.
******NEXT*******
about the world economic forum the world economic forum comm

******NEXT*******
pwc  global digital iq survey available at https www.pwc.com us en advisory services digital iq assets pwc digital iq report.pdf.
******NEXT*******
. tractica media release  artificial intelligence implementations have expanded to encompass discrete use cases across virtually all industries  august  available at https www.tractica.com newsroom press releases artificial intelligence implementations have expanded to encompass discrete use cases across virtually all industries.information about john sterman s climate simulation project at climate interactive website available at https www.climateinteractive.org..
******NEXT*******
. silver david et al. mastering the game of go without human knowledge  nature   october  available at https www.nature.com articles nature.
******NEXT*******
also see deepmind blog post alphago zero learning from scratch available at https deepmind.com blog alphago zero learning scratch.
******NEXT*******
harnessing artificial intelligence for

# Send PDF content for 'classification and audit' or send it straight to training.

In [183]:
audit_yes_no = widgets.Dropdown(
    options =  {"Send for Classification and Audit":"CLAS_AUDIT", "Send straight to training":"TRAIN"},
    description='Action PDF Content:',
    disabled=False,
    style = my_style,
    layout = my_layout
)

def yes_no_function(audit_yes_no_value ):
    print (audit_yes_no_value)
    

output = widgets.interactive_output(yes_no_function, {
            'audit_yes_no_value': audit_yes_no
            })

display (audit_yes_no, output)

Dropdown(description='Action PDF Content:', layout=Layout(width='500px'), options={'Send for Classification an…

Output()

In [190]:
for file, path in gathered_pdf_files_dict.items():# for name, age in dictionary.iteritems():  (for Python 2.x)
        if path == pdf_drop_down.value:
            print(file)

Harnessing_Artificial_Intelligence_for_the_Earth_report_2018.pdf


In [184]:
GIT_USER = git_user.value 
GIT_PSWD = git_pswd.value 
GIT_REPO = 'AIVC'


###############################ADD STRAIGHT TO TRAINING############################## 
#TRAIN_PATH = 'cp_wssc/Data/Iterative-models-building/Classification results/Conventions/Audited/'
###############IF WE ARE VERY SURE ALL SENTENCES ARE VALID FOR THE CONVENT###########


if (audit_yes_no.value == "TRAIN"):
    pdf_text_df_train = pd.DataFrame(list(zip(texts, data_provenance, convention,  old , new)),
                                     columns = ["text", "provenance", "convention", "old", "new"]) 
    display (pdf_text_df_train.head(5))
    ##Save the file directly to the training folder
    D, H = helper.time_stamp()
    file_name = 'audited_training_data_directPDF_' + D + '_' + H + '.tsv'
    
    #first save it to the local directory
    pdf_text_df.to_csv(LOCAL_PDF_DOCS_DIR + file_name, sep = '\t', index = False)
    f = open( os.path.join(LOCAL_PDF_DOCS_DIR, file_name), 'rb')
    f.close()
    
    #now push it to gitHub
    my_file_list = [LOCAL_PDF_DOCS_DIR + file_name]
    push_to_git_as = [TRAIN_PATH+file_name]
    
    commit, message = save_to_github(GIT_USER, GIT_PSWD, GIT_REPO, my_file_list, push_to_git_as)
    print ("saving to git as ", push_to_git_as[0])
    print ("Commit:", commit)
    print ("Message:", message)
       

        
#############################PUSH TO GITHUB############################## 
###############TO FOLLOW THE CLASSIFY AND AUDIT FLOW#####################

if (audit_yes_no.value == "CLAS_AUDIT"): 
#Save the file directly to the folder for audit AIVC/cp_wssc/Data/Iterative-models-building/Gathered_data/Conventions/

    pdf_text_df = pd.DataFrame(list(zip(texts, data_provenance)),
                columns = ("sentence", "provenance")) 
    
    D, H = helper.time_stamp()
    
    for file, path in gathered_pdf_files_dict.items():    # for name, age in dictionary.iteritems():  (for Python 2.x)
        if file == pdf_drop_down.value:
            print(file)
            
    file = file[:-4]
    file_name = 'gathered_pdf_' + convention + '_' + file + '.tsv'
    
    #first save it to the local directory
    pdf_text_df.to_csv(LOCAL_PDF_DOCS_DIR + file_name, sep = '\t', index = False)
    f = open( os.path.join(LOCAL_PDF_DOCS_DIR, file_name), 'rb')
    f.close()
    
    #now push it to gitHub
    my_file_list = [LOCAL_PDF_DOCS_DIR + file_name]
    push_to_git_as = [CLAS_AUDIT_PATH+file_name]
    
    commit, message = save_to_github(GIT_USER, GIT_PSWD, GIT_REPO, my_file_list, push_to_git_as)
    print ("saving to git as ", push_to_git_as[0])
    print ("Commit:", commit)
    print ("Message:", message)

TypeError: must be str, not list

In [165]:
document_lengths = np.array(list(map(len, pdf_text_df.text.str.split(' '))))

print("The average number of words in a sentence is: {}.".format(np.mean(document_lengths)))
print("The minimum number of words in a sentence is: {}.".format(min(document_lengths)))
print("The maximum number of words in a sentence is: {}.".format(max(document_lengths)))

The average number of words in a sentence is: 20.622222222222224.
The minimum number of words in a sentence is: 3.
The maximum number of words in a sentence is: 66.


In [167]:
!pip install PyPDF2
import PyPDF2

PDF = PyPDF2.PdfFileReader(memoryFile)

pages = PDF.getNumPages()
key = '/Annots'
uri = '/URI'
ank = '/A'

pdf_list = []

for page in range(pages):
    pageSliced = PDF.getPage(page)
    pageObject = pageSliced.getObject()
    if key in pageObject.keys():
        ann = pageObject[key]
        for a in ann:
            u = a.getObject()
            if uri in u[ank].keys():
                pdf_list.append(u[ank][uri])

pdf_list = keep_pdf_urls_only(pdf_list)
print ("additional pdf's found:", len(pdf_list))
for pdf_link in pdf_list:
    print(pdf_link)

You should consider upgrading via the 'pip install --upgrade pip' command.
additional pdf's found: 0


In [ ]:
http://www.uwyo.edu/barbier/publications/a global green new deal-executive summery.pdf